In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import pmdarima as pm
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
df = pd.read_csv(r"C:\Users\Robyi\Documents\Data Science Dataset\elecproduction.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.head()

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.set_index("date", inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df, label="Electricity Production", color="blue")
plt.xlabel("Time")
plt.ylabel("Production")
plt.title("Electricity Production Over Time")
plt.legend()
plt.show()

In [ ]:
model = pm.auto_arima(
    df, 
    seasonal=True,
    m=12,
    stepwise=True, 
    trace=True,
    error_action="ignore",
    suppress_warnings=True
)

In [ ]:
forecast, conf_int = model.predict(n_periods=1, return_conf_int=True)

print(f"🔹 Prediksi Produksi Listrik Periode Berikutnya: {forecast[0]:.2f}")
print(f"🔹 Interval Kepercayaan 95%: ({conf_int[0][0]:.2f}, {conf_int[0][1]:.2f})")

In [ ]:
def objective(trial):
    p = trial.suggest_int("p", 0, 5)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 5)
    P = trial.suggest_int("P", 0, 3)
    D = trial.suggest_int("D", 0, 1)
    Q = trial.suggest_int("Q", 0, 3)
    m = 12 

    try:
        model = SARIMAX(df, order=(p, d, q), seasonal_order=(P, D, Q, m))
        model_fit = model.fit(disp=False)
        aic = model_fit.aic
    except:
        aic = np.inf 
    
    return aic

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

In [ ]:
best_params = study.best_params
print(f"🔹 Best SARIMA Order: (p={best_params['p']}, d={best_params['d']}, q={best_params['q']})")
print(f"🔹 Best Seasonal Order: (P={best_params['P']}, D={best_params['D']}, Q={best_params['Q']}, m=12)")

In [ ]:
best_model = SARIMAX(
    df, 
    order=(best_params["p"], best_params["d"], best_params["q"]),
    seasonal_order=(best_params["P"], best_params["D"], best_params["Q"], 12)
)
best_model_fit = best_model.fit(disp=False)

In [ ]:
forecast_best = best_model_fit.forecast(steps=1)

print(f"🔹 Prediksi Terbaik Produksi Listrik: {forecast_best[0]:.2f}")